In [1]:
import numpy as np
import tflearn
import tflearn.activations as activations
# Data loading and preprocessing
from tflearn.activations import relu
from tflearn.data_utils import shuffle, to_categorical
from tflearn.layers.conv import avg_pool_2d, conv_2d, max_pool_2d
from tflearn.layers.core import dropout, flatten, fully_connected, input_data
from tflearn.layers.merge_ops import merge
from tflearn.layers.normalization import local_response_normalization
from tflearn.layers.normalization import batch_normalization


#chris library imports
from matplotlib import pyplot as plt
import cv2
import tensorflow as tf
from sklearn.model_selection import train_test_split

from test_recursive_image_load_V2 import load_CKP_data
from test_recursive_image_load_V2 import load_formated_data
from showNumpyInfo import showInfo


# global Paths to define for each specific computer
#tf_checkpoints = where the checkpoints of tensorflow training algorithms are stored to be recovered if necessary
tf_checkpoints = "G:/Documenten/personal/school/MaNaMA_AI/thesis/implementation/dexpression/github_1/DeXpression-master_chris_V3/tf_checkpoints"

# cascPath = the path to the cascade file for the facerecognition (relative paths didn't work on my windows edition)
cascPath = "G:/Documenten/personal/school/MaNaMA_AI/thesis/implementation/dexpression/github_1/DeXpression-master_chris/haarcascade.xml"

# Give a run ID here. Change it to flags (arguments) in version 2.
ID = '4_1'
RUNID = 'DeXpression_run_' + ID

# Give a dropout if required (change to True and define the dropout percentage).
dropout = False
dropout_keep_prob=0.5

# Load data
X_data = np.load('CKP_X.npy')
Y_data = np.load('CKP_Y.npy')

X_reduced = X_data[0:29]
Y_reduced = Y_data[0:29]

showInfo(X_reduced,"X_reduced")
showInfo(Y_reduced,"Y_reduced")


hdf5 is not supported on this machine (please install/reinstall h5py for optimal experience)
curses is not supported on this machine (please install/reinstall curses for an optimal experience)
Name  X_reduced
type:  <class 'numpy.ndarray'>
Dtype:  float64
shape:  (29, 224, 224, 1)
flags:    C_CONTIGUOUS : True
  F_CONTIGUOUS : False
  OWNDATA : False
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False
  UPDATEIFCOPY : False
Name  Y_reduced
type:  <class 'numpy.ndarray'>
Dtype:  float64
shape:  (29, 7)
flags:    C_CONTIGUOUS : True
  F_CONTIGUOUS : False
  OWNDATA : False
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False
  UPDATEIFCOPY : False


In [2]:


#-----------demostration ----------------
test_X = np.asarray([1,2,3,4,5,6,7,8,9,10])
test_Y = np.asarray([11,12,13,14,15,16,17,18,19,20])

print(test_X)
print(test_Y)

s = np.arange(test_X.shape[0])
print(s)
np.random.shuffle(s)

print(s)
print(test_X[s])
print(test_Y[s])


test_data = np.asarray([test_X,test_Y])

X_train, X_test, Y_train, Y_test = train_test_split(test_X, test_Y, test_size=0.33, random_state=42)
#Everytime you use 42, you will always get the same output the first time you make the split.

print("X_train = ",  X_train)
print("Y_train = ",  Y_train)
print("X_test = ",  X_test)
print("Y_test = ",  Y_test)
#-----------------------------------------


X = X_reduced.astype('uint8')
Y = Y_reduced

[ 1  2  3  4  5  6  7  8  9 10]
[11 12 13 14 15 16 17 18 19 20]
[0 1 2 3 4 5 6 7 8 9]
[5 1 0 4 9 2 8 6 7 3]
[ 6  2  1  5 10  3  9  7  8  4]
[16 12 11 15 20 13 19 17 18 14]
X_train =  [ 8  3 10  5  4  7]
Y_train =  [18 13 20 15 14 17]
X_test =  [9 2 6 1]
Y_test =  [19 12 16 11]


In [3]:
showInfo(X , "X")

cv2.imshow("example", X[1].reshape((224,224)))
cv2.waitKey(0)

print("size labels shape: " + repr(Y.shape))
print("type labels: " + repr(type(Y)))


Name  X
type:  <class 'numpy.ndarray'>
Dtype:  uint8
shape:  (29, 224, 224, 1)
flags:    C_CONTIGUOUS : True
  F_CONTIGUOUS : False
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False
  UPDATEIFCOPY : False
size labels shape: (29, 7)
type labels: <class 'numpy.ndarray'>


In [4]:
# Define number of output classes.
num_classes = 7

# Define padding scheme.
padding = 'VALID'

# Model Architecture
network = input_data(shape=[None, 224, 224, 1])
conv_1 = relu(conv_2d(network, 64, 7, strides=2, bias=True, padding=padding, activation=None, name='Conv2d_1'))
maxpool_1 = batch_normalization(max_pool_2d(conv_1, 3, strides=2, padding=padding, name='MaxPool_1'))
#LRN_1 = local_response_normalization(maxpool_1, name='LRN_1')
# FeatEX-1
conv_2a = relu(conv_2d(maxpool_1, 96, 1, strides=1, padding=padding, name='Conv_2a_FX1'))
maxpool_2a = max_pool_2d(maxpool_1, 3, strides=1, padding=padding, name='MaxPool_2a_FX1')
conv_2b = relu(conv_2d(conv_2a, 208, 3, strides=1, padding=padding, name='Conv_2b_FX1'))
conv_2c = relu(conv_2d(maxpool_2a, 64, 1, strides=1, padding=padding, name='Conv_2c_FX1'))
FX1_out = merge([conv_2b, conv_2c], mode='concat', axis=3, name='FX1_out')
# FeatEX-2
conv_3a = relu(conv_2d(FX1_out, 96, 1, strides=1, padding=padding, name='Conv_3a_FX2'))
maxpool_3a = max_pool_2d(FX1_out, 3, strides=1, padding=padding, name='MaxPool_3a_FX2')
conv_3b = relu(conv_2d(conv_3a, 208, 3, strides=1, padding=padding, name='Conv_3b_FX2'))
conv_3c = relu(conv_2d(maxpool_3a, 64, 1, strides=1, padding=padding, name='Conv_3c_FX2'))
FX2_out = merge([conv_3b, conv_3c], mode='concat', axis=3, name='FX2_out')
net = flatten(FX2_out)
if dropout:
    net = dropout(net, dropout_keep_prob)
loss = fully_connected(net, num_classes,activation='softmax')

# Compile the model and define the hyperparameters
network = tflearn.regression(loss, optimizer='Adam',
                     loss='categorical_crossentropy',
                     learning_rate=0.0001)

# Final definition of model checkpoints and other configurations
#model = tflearn.DNN(network, checkpoint_path='/home/cc/DeXpression/DeXpression_checkpoints',
model = tflearn.DNN(network, checkpoint_path=tf_checkpoints,
                    max_checkpoints=1, tensorboard_verbose=2, tensorboard_dir="./tflearn_logs/")

Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [5]:
# Fit the model, train for 20 epochs. (Change all parameters to flags (arguments) on version 2.)
model.fit(X, Y, n_epoch=20, validation_set=0.1, shuffle=True, show_metric=True, batch_size=350, snapshot_step=2000,snapshot_epoch=True, run_id=RUNID)

# Save the model
model.save(tf_checkpoints + '/' + RUNID + '.model')

# Load the model if required, later.
#model.load('./DeXpression_checkpoints/' + RUNID + '.model')

---------------------------------
Run id: DeXpression_run_4_1
Log directory: ./tflearn_logs/
INFO:tensorflow:Summary name Accuracy/ (raw) is illegal; using Accuracy/__raw_ instead.
---------------------------------
Training samples: 26
Validation samples: 3
--
Training Step: 1  | time: 13.954s
| Adam | epoch: 001 | loss: 0.00000 - acc: 0.0000 | val_loss: 15.35057 - val_acc: 0.3333 -- iter: 26/26
--
Training Step: 2  | total loss: 8.14023 | time: 5.703s
| Adam | epoch: 002 | loss: 8.14023 - acc: 0.1038 | val_loss: 13.47125 - val_acc: 0.3333 -- iter: 26/26
--


KeyboardInterrupt: 